## Setup

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

In [ ]:
from utils import apply_universe_filters

In [3]:
# Load universe definitions from YAML
import yaml

with open("universes.yaml") as f:
    universes = yaml.safe_load(f)

## 01 Data Checks

In [4]:
siab = pd.read_csv("./data/siab.csv")

In [ ]:
# Compute summary statistics
siab.describe(include = 'all')

In [ ]:
# Count number of missing values
siab.isna().sum()

In [5]:
grouped = siab.groupby('year')
siab_s = grouped.apply(lambda x: x.sample(n = 5000, random_state = 42)) # Sample 5000 obs from each year
siab_s = siab_s.reset_index(drop = True) # Ungroup

/var/folders/h7/6qcvyjh51cg86vrxn3xs8c_40000gn/T/ipykernel_7681/328774534.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  siab_s = grouped.apply(lambda x: x.sample(n = 5000, random_state = 42)) # Sample 5000 obs from each year


In [6]:
siab_s.groupby('year').describe(include = 'all')

persnr                              ... seeking1_tot_dur_byage                       
       count         mean            std  ...                    50%        75%         max
year                                      ...                                              
2010  5000.0  902794.9456  526060.681784  ...               9.350384  24.877980   98.466667
2011  5000.0  907358.6246  531695.182682  ...              10.500000  27.148810  107.000000
2012  5000.0  917696.5838  526272.709872  ...              10.932852  27.887941  116.567568
2013  5000.0  912578.6148  532244.457640  ...              10.773810  28.866667  114.000000
2014  5000.0  912746.6276  529512.009342  ...              10.287594  30.464228  134.638889
2015  5000.0  919043.7782  525640.947844  ...               9.991667  29.521542  121.486486
2016  5000.0  911624.4798  524165.653319  ...               8.469848  28.360736  118.904762

[7 rows x 1304 columns]

## 02 Data Split

Train with 2010 - 2014, calibrate with 2015, test with 2016

In [ ]:
siab_train = siab_s[siab_s.year < 2015] 
siab_calib = siab[siab.year == 2015] 
siab_test = siab[siab.year == 2016]

In [12]:
# Save

siab_train.to_csv('./data/siab_train.csv', index = False)
siab_calib.to_csv('./data/siab_calib.csv', index = False)
siab_test.to_csv('./data/siab_test.csv', index = False)

In [9]:
#new 30.6.

df_train = apply_universe_filters(siab_train, universes)

TypeError: list indices must be integers or slices, not str

In [ ]:
# Keep protected features, data from 2010-2014
X_train_f = siab_train.iloc[:,4:164]

In [ ]:
# Drop protected attributes, data from 2010-2014
X_train_s = X_train_f.drop(
    columns = ['frau1', 'maxdeutsch1', 'maxdeutsch.Missing.'])

In [ ]:
# ltue, from siab_train, i.e. 2010-2014
y_train = siab_train.iloc[:, [3]]

In [ ]:
# Keep protected features, data from 2015
X_calib_f = siab_calib.iloc[:,4:164]

# Drop protected features, data from 2015
X_calib_s = X_calib_f.drop(
    columns = ['frau1', 'maxdeutsch1', 'maxdeutsch.Missing.'])

# ltue, from siab_calib, i.e. 2015
y_calib = siab_calib.iloc[:, [3]]

In [ ]:
# Keep protected features, data from 2016
X_test_f = siab_test.iloc[:,4:164]

# Drop protected features, data from 2016
X_test_s = X_test_f.drop(
    columns = ['frau1', 'maxdeutsch1', 'maxdeutsch.Missing.'])

# ltue, from siab_test, i.e. 2016
y_test = siab_test.iloc[:, [3]]

## 03 Descriptive Stats

In [ ]:
siab_t = siab_train.copy(deep = True)
siab_t = pd.concat([siab_t, siab_calib, siab_test], ignore_index=True)

In [ ]:
siab_t['nongerman'] = np.where(siab_t['maxdeutsch1'] == 0, 1, 0) # creates new column 'nongerman', if maxdeutsch1 == 0 then nongerman = 1, else 0 
siab_t.loc[siab_t['maxdeutsch.Missing.'] == 1, 'nongerman'] = np.nan # overwrite nongerman with NaN for any row where maxdeutsch.Missing. equals 1
siab_t['nongerman_male'] = np.where((siab_t['nongerman'] == 1) & (siab_t['frau1'] == 0), 1, 0)
siab_t['nongerman_female'] = np.where((siab_t['nongerman'] == 1) & (siab_t['frau1'] == 1), 1, 0)

In [ ]:
desc1 = siab_t[['year', 'ltue']].groupby('year').mean()
desc1.to_latex('./output/desc1.tex', float_format = "%.3f") # Mean LTUE over time

In [ ]:
desc2a = siab_t[['year', 'frau1', 'nongerman', 'nongerman_male', 'nongerman_female']].groupby(['year']).agg(['sum', 'count'])
desc2b = siab_t[['year', 'frau1', 'nongerman', 'nongerman_male', 'nongerman_female']].groupby(['year']).mean()
desc2c = siab_t[['year', 'ltue', 'frau1', 'nongerman', 'nongerman_male', 'nongerman_female']].groupby(['year', 'ltue']).agg(['mean', 'count'])

In [ ]:
desc2a.to_latex('./output/desc2a.tex', float_format = "%.3f") # Number of cases over time
desc2b.to_latex('./output/desc2b.tex', float_format = "%.3f") # Socio-demo over time
desc2c.to_latex('./output/desc2c.tex', float_format = "%.3f") # Socio-demo by LTUE over time

## Save 

In [ ]:
X_train_f.to_csv('./output/X_train_f.csv', index = False)
X_train_s.to_csv('./output/X_train_s.csv', index = False)
y_train.to_csv('./output/y_train.csv', index = False)

X_calib_f.to_csv('./output/X_calib_f.csv', index = False)
X_calib_s.to_csv('./output/X_calib_s.csv', index = False)
y_calib.to_csv('./output/y_calib.csv', index = False)

X_test_f.to_csv('./output/X_test_f.csv', index = False)
X_test_s.to_csv('./output/X_test_s.csv', index = False)
y_test.to_csv('./output/y_test.csv', index = False)